# RAG 시스템 - 뷰티 코칭 AI (Hybrid: HuggingFace + OpenAI)

이 노트북은 PDF 문서를 기반으로 한 RAG(Retrieval-Augmented Generation) 시스템입니다.

## 하이브리드 접근 (비용 효율적!) ⭐
- **임베딩**: HuggingFace multilingual-e5-large (무료, GPU/CPU, 다국어 지원)
- **LLM**: OpenAI GPT-4o-mini (유료, 고품질, 비전 지원)

## 주요 기능
- PDF 문서 로드 및 전처리
- HuggingFace 다국어 임베딩 및 Chroma DB 저장 (무료!)
- 한글 쿼리로 영어 논문도 검색 가능
- OpenAI GPT-4o-mini 비전 모델을 사용한 이미지+텍스트 질의응답

## 1. 라이브러리 Import




In [1]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
import re
import json
import time
import os
import base64
from pathlib import Path

load_dotenv()

# OpenAI API 키 확인
if not os.getenv("OPENAI_API_KEY"):
    print("❌ OPENAI_API_KEY가 설정되지 않았습니다. .env 파일을 확인하세요.")
else:
    print("✅ OpenAI API 키가 설정되었습니다.")

✅ OpenAI API 키가 설정되었습니다.


## 2. 유틸리티 함수 정의




In [ ]:
def clean_text(text):
    """텍스트 공백 정리 (PDF OCR 오류 대응 강화)"""
    
    # 1. 단어 내부의 과도한 공백 제거 (가장 공격적)
    # "T r a d i t i o n a l" -> "Traditional"
    # 패턴: 1-2글자 + 공백이 3번 이상 반복
    text = re.sub(r'\b(\w{1,2})\s+(?=\w{1,2}\s+\w)', r'\1', text)
    
    # 2. 남은 단일 문자 사이 공백 제거
    # "t h e" -> "the"
    text = re.sub(r'\b(\w)\s+(\w)\s+(\w)\b', r'\1\2\3', text)
    text = re.sub(r'\b(\w)\s+(\w)\b', r'\1\2', text)
    
    # 3. 특수문자와 단어 사이 불필요한 공백 제거
    # "ﬂ a t" -> "flat"
    text = re.sub(r'(\S)\s+(\S)(?=\s+\S{1,2}\s+|\s+\S{1,2}[,.])', r'\1\2', text)
    
    # 4. 여러 개의 연속된 공백을 하나로
    text = re.sub(r' {2,}', ' ', text)
    
    # 5. 탭을 공백으로
    text = text.replace('\t', ' ')
    
    # 6. 줄바꿈 전후 하이픈 처리 (단어 분리)
    # "dis-\nrupting" -> "disrupting"
    text = re.sub(r'-\s*\n\s*', '', text)
    
    # 7. 3개 이상의 연속된 줄바꿈을 2개로
    text = re.sub(r'\n{3,}', '\n\n', text)
    
    # 8. 각 줄의 앞뒤 공백 제거
    lines = [line.strip() for line in text.split('\n')]
    
    # 9. 빈 줄과 너무 짧은 줄 제거 (노이즈 제거)
    lines = [line for line in lines if len(line) > 1]
    
    text = '\n'.join(lines)
    
    return text.strip()

def clean_mixed_content(text):
    """본문과 섞인 참고문헌 정리"""
    
    # 참고문헌 섹션 전체 제거 (가장 우선)
    # "\n참고문헌\n" 이후 모든 내용 제거
    text = re.sub(r'\n참고문헌\n[\s\S]*$', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\n참 고 문 헌\n[\s\S]*$', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\nReferences?\n[\s\S]*$', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\n참고 문헌\n[\s\S]*$', '', text, flags=re.IGNORECASE)
    
    # 참고문헌 패턴 (본문 중간에 섞인 경우)
    reference_patterns = [
        # 학위논문 형식: 번호) 저자.(연도). 제목... 대학원, 학위논문, pp.숫자
        r'\d+\)\s+[^.\n]+\.\(\d{4}\)\.[\s\S]*?(?:대학원|大學院)[\s\S]*?(?:석사|박사|碩士|博士)[\s\S]*?pp?\.\s*\d+[-\d]*\.',
        
        # 일반 참고문헌 형식 (op. cit. 포함)
        r'\d+\)\s+[^.\n]+\.\(\d{4}\)\.[\s\S]*?(?:op\.\s*cit\.|ibid\.)[\s\S]*?p\.\d+\.',
        
        # 저널 형식: 저자. 제목. 저널명, 번호: 페이지, 연도.
        r'[가-힣a-zA-Z\s,]+\.\s*[^.\n]+\.\s*[^,\n]+,\s*\d+:\s*\d+[-\d]*,\s*\d{4}\.',
        
        # 기본 참고문헌 형식
        r'\d+\)\s+[^.(]+\.\(\d{4}\)\.[^\n]*(?:\n(?!\d+\))[^\n]+)*',
    ]
    
    cleaned = text
    for pattern in reference_patterns:
        cleaned = re.sub(pattern, '', cleaned, flags=re.IGNORECASE)
    
    # 연속된 빈 줄 정리
    cleaned = re.sub(r'\n\s*\n\s*\n+', '\n\n', cleaned)
    
    return cleaned.strip()

def load_single_pdf(pdf_path):
    """단일 PDF 파일 로드 (PyMuPDFLoader 사용)"""
    try:
        loader = PyMuPDFLoader(str(pdf_path))
        docs = loader.load()
        return docs, "PyMuPDFLoader"
    except Exception as e:
        # 에러 정보를 함께 반환
        raise Exception(f"PyMuPDFLoader 실패: {str(e)}")

def load_documents():
    """PDF 문서 로드"""
    
    # data 폴더의 모든 PDF 파일 찾기
    pdf_files = list(Path("data/").rglob("*.pdf"))
    print(f"📁 발견된 PDF 파일: {len(pdf_files)}개\n")
    
    all_documents = []
    failed_files = []
    loader_stats = {}
    
    for pdf_path in pdf_files:
        try:
            print(f"  로딩: {pdf_path.name}...", end=" ")
            docs, loader_name = load_single_pdf(pdf_path)
            all_documents.extend(docs)
            loader_stats[loader_name] = loader_stats.get(loader_name, 0) + 1
            print(f"✅ ({len(docs)}페이지, {loader_name})")
        except Exception as e:
            print(f"❌ 실패: {str(e)}")
            failed_files.append((pdf_path.name, str(e)))
    
    print(f"\n✅ 총 {len(all_documents)}개 페이지 로드 완료")
    print(f"📊 사용된 로더: {loader_stats}")
    
    if failed_files:
        print(f"\n⚠️ {len(failed_files)}개 파일 로드 실패:")
        for filename, error in failed_files:
            print(f"  - {filename}: {error}")
    
    # 텍스트 정리 적용
    print("\n🧹 텍스트 정리 중...")
    for i, doc in enumerate(all_documents):
        original_length = len(doc.page_content)
        doc.page_content = clean_text(doc.page_content)
        cleaned_length = len(doc.page_content)
        
        # 정리 효과 확인 (첫 페이지만)
        if i == 0 and original_length > 0:
            print(f"\n📊 첫 페이지 정리 효과:")
            print(f"  원본 길이: {original_length}자")
            print(f"  정리 후: {cleaned_length}자")
            print(f"  감소율: {(1 - cleaned_length/original_length)*100:.1f}%")
            print(f"\n첫 200자 미리보기:")
            print(doc.page_content[:200])
            print("...")
    
    return all_documents

def chunk_doc(documents, chunk_size=1000, chunk_overlap=200):
    """문서를 청크로 분할 - 개선된 설정
    
    Args:
        chunk_size: 청크 크기 (기본 1000자 - 더 큰 의미 단위 유지)
        chunk_overlap: 청크 간 중복 (기본 200자 - 문맥 연속성 유지)
    """
    # clean_mixed_content 적용
    print("\n🧹 참고문헌 정리 중...")
    for i, doc in enumerate(documents):
        original_length = len(doc.page_content)
        doc.page_content = clean_mixed_content(doc.page_content)
        cleaned_length = len(doc.page_content)
        
        # 정리 효과 확인 (첫 문서만)
        if i == 0 and original_length > 0:
            print(f"\n📊 첫 문서 참고문헌 정리 효과:")
            print(f"  원본 길이: {original_length}자")
            print(f"  정리 후: {cleaned_length}자")
            print(f"  감소율: {(1 - cleaned_length/original_length)*100:.1f}%")
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        # separators 순서 개선: 큰 의미 단위부터 분할
        separators=[
            "\n\n\n",  # 여러 단락 구분
            "\n\n",    # 단락 구분
            "\n",      # 줄바꿈
            "。",      # 한글 마침표
            ". ",      # 영문 문장 끝
            "! ",      # 느낌표
            "? ",      # 물음표
            "；",      # 세미콜론
            "，",      # 쉼표
            " ",       # 공백 (최후의 수단)
            ""         # 문자 단위 (최후의 최후)
        ],
        keep_separator=True,  # 구분자 유지
        length_function=len,
    )
    splits = text_splitter.split_documents(documents)
    
    # 청크 품질 확인
    print(f"\n📊 청크 통계:")
    print(f"  총 청크 수: {len(splits)}")
    if len(splits) > 0:
        avg_length = sum(len(s.page_content) for s in splits) / len(splits)
        min_length = min(len(s.page_content) for s in splits)
        max_length = max(len(s.page_content) for s in splits)
        print(f"  평균 청크 길이: {avg_length:.0f}자")
        print(f"  최소/최대: {min_length}/{max_length}자")
        print(f"\n첫 번째 청크 미리보기 (500자):")
        print(splits[0].page_content[:500])
        print("...")
    
    return splits

def save_documents(documents, output_path):
    """문서를 JSONL 형식으로 저장"""
    with open(output_path, 'w', encoding='utf-8') as f:
        for i, page in enumerate(documents):
            data = {
                'page': i + 1,
                'text': page.page_content,
                'metadata': {
                    'source': page.metadata.get('source', 'unknown'),
                    'page': page.metadata.get('page', i)
                }
            }
            f.write(json.dumps(data, ensure_ascii=False) + '\n')

def encode_image(image_path):
    """이미지를 base64로 인코딩"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

print("✅ 유틸리티 함수 정의 완료 (개선된 청킹 설정)")

## 3. 문서 로드 및 전처리

**이 셀은 한 번만 실행하세요!**

In [3]:
print("문서 로드 중...")
documents = load_documents()
print(f"✅ 총 {len(documents)}개의 문서 로드 완료")

print("\n문서 청킹 중...")
chunked_documents = chunk_doc(documents)
print(f"✅ 총 {len(chunked_documents)}개의 청크 생성 완료")

print("\n문서 저장 중...")
save_documents(chunked_documents, 'tmp.jsonl')
print("✅ tmp.jsonl 저장 완료")

문서 로드 중...
📁 발견된 PDF 파일: 171개

  로딩: 헤어스타일 변화로 인한 감정 치유와 회복 탄력성의 관계에서 자존감의 매개 효과.pdf... ✅ (14페이지, PyMuPDFLoader)
  로딩: 안면 노화 징후 완화를 위한 구강안면 근기능 개입의 효과-PubMed_Central.pdf... ✅ (24페이지, PyMuPDFLoader)
  로딩: 외모·자아수용·퍼펙셔니즘.pdf... ✅ (8페이지, PyMuPDFLoader)
  로딩: 젊음을 위한 얼굴 운동-Scielo.pdf... ✅ (12페이지, PyMuPDFLoader)
  로딩: 석곡및병풀복합추출물의피부색소침착방지및피부재생효과.pdf... ✅ (9페이지, PyMuPDFLoader)
  로딩: 남성 헤어커트의 작품제작.pdf... ✅ (18페이지, PyMuPDFLoader)
  로딩: 진여수클린미스트를 이용한 중년여성의 안면피부개선 효과.pdf... ✅ (9페이지, PyMuPDFLoader)
  로딩: 성격 인상 정보가 얼굴 매력 판단에 미치는 영향.pdf... ✅ (18페이지, PyMuPDFLoader)
  로딩: JKSC-2025-31-2-451.pdf... ✅ (12페이지, PyMuPDFLoader)
  로딩: 미용복지서비스품질이 노인 심리적행복감과 자아실현감에 미치는 영향.pdf... ✅ (10페이지, PyMuPDFLoader)
  로딩: 저분자 펩타이드 콤플렉스를 포함한 마이크로 스피큘 화장품의 피부 개선 효과.pdf... ✅ (18페이지, PyMuPDFLoader)
  로딩: 모던걸 헤어스타일의 구성요소와 사회적 여성성과의 상관성 연구 - 미용학 전공 여대생 관점으로.pdf... ✅ (6페이지, PyMuPDFLoader)
  로딩: From Basic Science to Basic Skin Care copy.pdf... ✅ (9페이지, PyMuPDFLoader)
  로딩: 호호바오일의모발보호효과연구.pdf... ✅ (21페이지, PyMuPDFLoader)
  로딩: Benefi

## 4. 벡터 임베딩 및 저장 (HuggingFace multilingual-e5-large)

**이 셀도 한 번만 실행하세요! 이미 임베딩이 있다면 5번 셀로 바로 가세요.**

**HuggingFace 다국어 모델 사용 - 무료!** ⭐
**한글 쿼리로 영어 논문도 검색 가능!**

In [ ]:
print("=" * 50)
print("HuggingFace multilingual-e5-large 임베딩 시작...")
print(f"청크 개수: {len(chunked_documents)}개")
print("⭐ HuggingFace 다국어 모델 사용 - 무료!")
print("⭐ 한글 쿼리로 영어 논문도 검색 가능!")
print("=" * 50)

embedding_start = time.time()


# HuggingFace multilingual-e5-large 임베딩 모델 사용
# 다국어 지원으로 한글 쿼리로 영어 문서 검색 가능
embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large",
    model_kwargs={'device': 'cuda'},  # GPU 사용 (없으면 'cpu'로 변경)
    encode_kwargs={'normalize_embeddings': True}
)

print("✅ 임베딩 모델 로드 완료")
print("📥 벡터 임베딩 생성 중... (시간이 다소 걸릴 수 있습니다)")

vectorstore = Chroma.from_documents(
    documents=chunked_documents,
    embedding=embeddings,
    persist_directory="./chroma_db_tmp"
)

embedding_end = time.time()
print(f"✅ HuggingFace multilingual-e5-large 임베딩 완료: {embedding_end - embedding_start:.2f}초")
print("=" * 50)

## 5. 기존 벡터스토어 로드 (HuggingFace multilingual-e5-large)

**임베딩을 다시 하지 않고 기존 것을 사용하려면 이 셀을 실행하세요!**

**HuggingFace 다국어 모델 - 무료!** ⭐

In [ ]:
# if os.path.exists("./chroma_db_tmp"):
#     print("기존 HuggingFace multilingual-e5-large 벡터스토어 로드 중...")
    
#     embeddings = HuggingFaceEmbeddings(
#         model_name="intfloat/multilingual-e5-large",
#         model_kwargs={'device': 'cuda'},  # GPU 사용 (없으면 'cpu'로 변경)
#         encode_kwargs={'normalize_embeddings': True}
#     )
    
#     vectorstore = Chroma(
#         persist_directory="./chroma_db_tmp",
#         embedding_function=embeddings
#     )
#     print("✅ 벡터스토어 로드 완료")
# else:
#     print("❌ 벡터스토어가 없습니다. 4번 셀을 먼저 실행하세요.")

기존 HuggingFace multilingual-e5-large 벡터스토어 로드 중...


/tmp/ipykernel_9795/1055643921.py:10: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(


✅ 벡터스토어 로드 완료


## 6. 모델 설정




In [6]:
# OpenAI 모델 설정
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7
)

vision_llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

query_llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

print("✅ OpenAI 모델 설정 완료")
print("   - llm: 최종 답변 생성용")
print("   - vision_llm: 이미지 분석용")
print("   - query_llm: 검색 쿼리 최적화용")

✅ OpenAI 모델 설정 완료
   - llm: 최종 답변 생성용
   - vision_llm: 이미지 분석용
   - query_llm: 검색 쿼리 최적화용


## 7. 이미지 분석 및 검색 쿼리 생성 함수 (통합)




In [7]:
def analyze_image_and_create_query(image_base64, user_query):
    """이미지 분석과 검색 쿼리 생성을 한 번의 API 호출로 처리"""
    
    combined_prompt = f"""당신은 이미지 분석 및 학술 논문 검색 전문가입니다.

**단계 1: 이미지 분석**
제공된 이미지를 분석하여 다음 카테고리별로 관찰 사항을 작성하세요:

1. **헤어 상태**: 헤어 스타일, 길이, 볼륨, 질감, 색상, 광택, 손상 여부, 두피 상태, 전반적인 헤어 컨디션
2. **피부 상태**: 피부 톤, 색상, 수분감, 광택, 결, 텍스처, 특이사항 (잡티, 붉은기 등)
3. **얼굴 윤곽**: 얼굴형, 눈/코/입의 비율 및 특징, 턱선, 볼살, 전반적인 균형감

**단계 2: 검색 쿼리 생성**
사용자 질문과 이미지 분석 결과를 바탕으로, 헤어/피부/미용/뷰티 관련 학술 논문 및 전문 정보를 효과적으로 검색할 수 있는 최적화된 쿼리를 생성하세요.

[사용자 질문]: {user_query}

**출력 형식:**
아래 JSON 형식으로 출력하세요.

```json
{{
  "image_analysis": {{
    "hair": "헤어 상태 분석 (3-5문장)",
    "skin": "피부 상태 분석 (3-5문장)",
    "contour": "얼굴 윤곽 분석 (3-5문장)"
  }},
  "search_query": "학술 논문 검색에 최적화된 쿼리 (200-300자, 한국어와 영어 키워드 포함)"
}}
```

**주의사항:**
- 이미지 분석은 객관적이고 부정적 표현을 피할 것
- 검색 쿼리는 학술적/전문적 용어를 사용하여 검색 정확도를 높일 것"""

    message = HumanMessage(
        content=[
            {"type": "text", "text": combined_prompt},
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image_base64}"}
            }
        ]
    )
    
    response = vision_llm.invoke([message])
    
    # JSON 파싱
    import json
    import re
    
    # JSON 코드 블록 추출
    content = response.content
    json_match = re.search(r'```json\s*(\{.*?\})\s*```', content, re.DOTALL)
    
    if json_match:
        result = json.loads(json_match.group(1))
    else:
        # JSON 블록이 없으면 전체를 JSON으로 파싱 시도
        result = json.loads(content)
    
    return result


print("✅ 이미지 분석 및 검색 쿼리 생성 함수 정의 완료 (통합)")
print("   - analyze_image_and_create_query(image_base64, user_query)")
print("   - 한 번의 API 호출로 이미지 분석 + 검색 쿼리 생성")

✅ 이미지 분석 및 검색 쿼리 생성 함수 정의 완료 (통합)
   - analyze_image_and_create_query(image_base64, user_query)
   - 한 번의 API 호출로 이미지 분석 + 검색 쿼리 생성


## 8. Retriever 설정



In [10]:
from langchain_community.retrievers import BM25Retriever
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda

# Dense Retriever (벡터 유사도)
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

# Sparse Retriever (BM25 키워드)
bm25_retriever = BM25Retriever.from_documents(chunked_documents)
bm25_retriever.k = 5

print("✅ Retriever 설정 완료")
print("   - Dense Retriever (벡터 유사도): k=5")
print("   - Sparse Retriever (BM25): k=5")

✅ Retriever 설정 완료
   - Dense Retriever (벡터 유사도): k=5
   - Sparse Retriever (BM25): k=5


## 9. MultiQuery 설정




In [11]:
# MultiQuery: 하나의 질문을 3개의 다양한 검색 쿼리로 확장
multi_query_prompt = PromptTemplate.from_template("""
Generate 3 different search queries that help retrieve relevant academic or scientific information 
related to the following question. 
Write each query on a new line.

Question: {question}
""")

generate_queries = (
    multi_query_prompt
    | ChatOpenAI(model="gpt-4o-mini", temperature=0)
    | RunnableLambda(lambda x: x.content.strip().split("\n"))
)

print("✅ MultiQuery 설정 완료")
print("   - 1개의 질문 → 3개의 다양한 검색 쿼리로 확장")

✅ MultiQuery 설정 완료
   - 1개의 질문 → 3개의 다양한 검색 쿼리로 확장


## 10. RRF (Reciprocal Rank Fusion) 함수




In [12]:
def rrf_merge(dense_docs, sparse_docs, k=60):
    """RRF를 사용하여 Dense와 Sparse 검색 결과를 병합"""
    scores = {}

    def add_results(docs):
        for rank, doc in enumerate(docs):
            doc_id = doc.metadata.get("id", id(doc))
            score = 1 / (k + rank + 1)
            if doc_id not in scores:
                scores[doc_id] = {"doc": doc, "score": 0}
            scores[doc_id]["score"] += score

    add_results(dense_docs)
    add_results(sparse_docs)

    merged = sorted(scores.values(), key=lambda x: x["score"], reverse=True)
    return [item["doc"] for item in merged]


print("✅ RRF 함수 정의 완료")
print("   - rrf_merge(dense_docs, sparse_docs, k=60)")

✅ RRF 함수 정의 완료
   - rrf_merge(dense_docs, sparse_docs, k=60)


## 11. Hybrid Multi Retriever




In [13]:
def hybrid_multi_retrieve(queries):
    """MultiQuery로 생성된 여러 쿼리로 Hybrid 검색 수행"""
    dense_all = []
    sparse_all = []

    for q in queries:
        dense_all.extend(retriever.invoke(q))
        sparse_all.extend(bm25_retriever.invoke(q))

    merged = rrf_merge(dense_all, sparse_all)
    return merged[:10]  # 상위 10개만 반환


hybrid_multi_retriever = (
    generate_queries
    | RunnableLambda(hybrid_multi_retrieve)
)

print("✅ Hybrid Multi Retriever 설정 완료")
print("   - MultiQuery (3개 쿼리) + Dense + Sparse + RRF")
print("   - 최종 상위 10개 문서 반환")

✅ Hybrid Multi Retriever 설정 완료
   - MultiQuery (3개 쿼리) + Dense + Sparse + RRF
   - 최종 상위 10개 문서 반환


## 12. QA Chain 설정



## 13. 문서 처리 함수




In [14]:
def deduplicate_docs(docs):
    """중복 문서 제거"""
    seen = set()
    unique_docs = []
    for doc in docs:
        content_hash = hash(doc.page_content)
        if content_hash not in seen:
            seen.add(content_hash)
            unique_docs.append(doc)
    return unique_docs[:7]  # 최종적으로 7개만 반환


def format_docs(docs):
    """중복 제거 후 포맷팅"""
    unique_docs = deduplicate_docs(docs)
    return "\n\n".join(doc.page_content for doc in unique_docs)


print("✅ 문서 처리 함수 정의 완료")
print("   - deduplicate_docs(docs): 중복 제거, 상위 7개")
print("   - format_docs(docs): 문서 포맷팅")

✅ 문서 처리 함수 정의 완료
   - deduplicate_docs(docs): 중복 제거, 상위 7개
   - format_docs(docs): 문서 포맷팅


In [15]:
# QA 프롬프트 템플릿
qa_prompt = PromptTemplate.from_template("""당신은 헤어/피부/미용 분야의 전문 코칭 AI입니다.

제공된 학술 논문 및 전문 자료를 바탕으로 사용자의 질문에 답변하세요.

**답변 형식 (JSON):**
```json
{{
  "Appearance_Coaching": {{
    "Hair": {{
      "status": "현재 헤어 상태 분석 (관련 연구 결과 인용)",
      "improvement_tips": ["구체적인 개선 팁 1", "구체적인 개선 팁 2"]
    }},
    "Skin": {{
      "status": "현재 피부 상태 분석 (관련 연구 결과 인용)",
      "improvement_tips": ["구체적인 개선 팁 1", "구체적인 개선 팁 2"]
    }},
    "Contour": {{
      "status": "현재 얼굴 윤곽 분석 (관련 연구 결과 인용)",
      "improvement_tips": ["구체적인 개선 팁 1", "구체적인 개선 팁 2"]
    }}
  }}
}}
```

**지침:**
1. 제공된 자료에서 관련 연구 결과를 인용하세요
2. 구체적이고 실행 가능한 개선 팁을 제공하세요
3. 부정적인 표현은 피하고 긍정적으로 작성하세요
4. 모든 조언은 학술적 근거를 바탕으로 하세요

**참고 자료:**
{context}

**질문:**
{question}

**답변:**""")

# RAG Chain 구성
qa_chain = (
    {
        "context": hybrid_multi_retriever | RunnableLambda(format_docs),
        "question": RunnableLambda(lambda x: x)
    }
    | qa_prompt
    | llm
    | StrOutputParser()
)

print("✅ QA Chain 설정 완료")
print("   - hybrid_multi_retriever + format_docs + qa_prompt + llm")

✅ QA Chain 설정 완료
   - hybrid_multi_retriever + format_docs + qa_prompt + llm


In [16]:
def get_dense_retriever_with_scores(query, k=5):
    """Dense Retriever (벡터 유사도)로 검색하고 점수와 함께 반환"""
    results = vectorstore.similarity_search_with_score(query, k=k)
    return [{"document": doc, "score": score} for doc, score in results]


def get_bm25_retriever_with_scores(query, k=5):
    """BM25 Retriever (키워드 기반)로 검색하고 점수와 함께 반환"""
    # BM25Retriever는 내부적으로 점수를 계산하지만 직접 접근이 어려움
    # 대신 rank를 기반으로 점수를 계산 (높을수록 좋음)
    docs = bm25_retriever.invoke(query)[:k]
    # Rank 기반 점수: 1위는 k점, 2위는 k-1점...
    return [{"document": doc, "score": k - i} for i, doc in enumerate(docs)]


def rrf_merge_with_scores(dense_docs, sparse_docs, k=60):
    """RRF를 사용하여 Dense와 Sparse 검색 결과를 병합하고 점수 반환"""
    scores = {}

    def add_results(docs):
        for rank, doc in enumerate(docs):
            doc_id = doc.metadata.get("id", id(doc))
            score = 1 / (k + rank + 1)
            if doc_id not in scores:
                scores[doc_id] = {"doc": doc, "score": 0}
            scores[doc_id]["score"] += score

    add_results(dense_docs)
    add_results(sparse_docs)

    merged = sorted(scores.values(), key=lambda x: x["score"], reverse=True)
    return [{"document": item["doc"], "score": item["score"]} for item in merged]


def print_rrf_results(results, top_k=7):
    """RRF 병합 결과를 보기 좋게 출력"""
    print(f"\n📊 RRF 병합 결과 (상위 {top_k}개):")
    print("=" * 80)
    for i, result in enumerate(results[:top_k], 1):
        print(f"\n[{i}] RRF Score: {result['score']:.4f}")
        print(f"Content Preview: {result['document'].page_content[:200]}...")
        print(f"Source: {result['document'].metadata.get('source', 'N/A')}")
        print("-" * 80)


def get_all_retrievers_with_scores(query, k=5):
    """모든 retriever의 결과를 한 번에 확인"""
    print("=" * 80)
    print(f"테스트 쿼리: {query}")
    print("=" * 80)
    
    # Dense Retriever
    print("\n📊 1. Dense Retriever (Vector Similarity):")
    print("-" * 80)
    dense_results = get_dense_retriever_with_scores(query, k=k)
    for i, result in enumerate(dense_results, 1):
        print(f"[{i}] Score: {result['score']:.4f} | Preview: {result['document'].page_content[:100]}...")
    
    # BM25 Retriever
    print("\n\n📊 2. BM25 Retriever (Keyword-based):")
    print("-" * 80)
    sparse_results = get_bm25_retriever_with_scores(query, k=k)
    for i, result in enumerate(sparse_results, 1):
        print(f"[{i}] Score: {result['score']:.4f} | Preview: {result['document'].page_content[:100]}...")
    
    # RRF 병합
    print("\n\n📊 3. RRF 병합 결과:")
    print("-" * 80)
    dense_docs = [r['document'] for r in dense_results]
    sparse_docs = [r['document'] for r in sparse_results]
    rrf_results = rrf_merge_with_scores(dense_docs, sparse_docs, k=60)
    for i, result in enumerate(rrf_results[:7], 1):
        print(f"[{i}] RRF Score: {result['score']:.4f} | Preview: {result['document'].page_content[:100]}...")
    
    return {
        "dense": dense_results,
        "sparse": sparse_results,
        "rrf": rrf_results
    }


print("✅ Retriever 점수 확인 함수 정의 완료")
print("   - get_dense_retriever_with_scores(query, k)")
print("   - get_bm25_retriever_with_scores(query, k)")
print("   - rrf_merge_with_scores(dense_docs, sparse_docs, k)")
print("   - print_rrf_results(results, top_k)")
print("   - get_all_retrievers_with_scores(query, k)")

✅ Retriever 점수 확인 함수 정의 완료
   - get_dense_retriever_with_scores(query, k)
   - get_bm25_retriever_with_scores(query, k)
   - rrf_merge_with_scores(dense_docs, sparse_docs, k)
   - print_rrf_results(results, top_k)
   - get_all_retrievers_with_scores(query, k)


## 14. 테스트 및 실행




## 15. Retriever 점수 확인 함수




In [17]:
test_query = "I have alopecia areata and it’s affecting my confidence. Please suggest ways to improve both my hair condition and self-esteem."
hybrid_multi_retriever.invoke(test_query)

#get_dense_retriever_with_scores(test_query, k = 5)
# get_bm25_retriever_with_scores(test_query, k=5)
get_all_retrievers_with_scores(test_query, k=5)

테스트 쿼리: I have alopecia areata and it’s affecting my confidence. Please suggest ways to improve both my hair condition and self-esteem.

📊 1. Dense Retriever (Vector Similarity):
--------------------------------------------------------------------------------
[1] Score: 0.2802 | Preview: 1040-50.
7.
Lauron S, Plasse C, Vaysset M, Pereira B, D’Incan M, Rondepierre F, et al.
Prevalence an...
[2] Score: 0.2802 | Preview: 1040-50.
7.
Lauron S, Plasse C, Vaysset M, Pereira B, D’Incan M, Rondepierre F, et al.
Prevalence an...
[3] Score: 0.2802 | Preview: 1040-50.
7.
Lauron S, Plasse C, Vaysset M, Pereira B, D’Incan M, Rondepierre F, et al.
Prevalence an...
[4] Score: 0.2802 | Preview: 1040-50.
7.
Lauron S, Plasse C, Vaysset M, Pereira B, D’Incan M, Rondepierre F, et al.
Prevalence an...
[5] Score: 0.2802 | Preview: 1040-50.
7.
Lauron S, Plasse C, Vaysset M, Pereira B, D’Incan M, Rondepierre F, et al.
Prevalence an...


📊 2. BM25 Retriever (Keyword-based):
------------------------------------

{'dense': [{'document': Document(metadata={'creator': 'Adobe InDesign 19.5 (Macintosh)', 'modDate': "D:20251009095737+03'00'", 'keywords': '', 'trapped': '', 'creationdate': '2025-10-09T09:57:37+03:00', 'author': '', 'total_pages': 5, 'producer': 'Adobe PDF Library 17.0', 'creationDate': "D:20251009095737+03'00'", 'format': 'PDF 1.4', 'file_path': 'data/AA–불안·우울 연관 메타분석.pdf', 'source': 'data/AA–불안·우울 연관 메타분석.pdf', 'moddate': '2025-10-09T09:57:37+03:00', 'subject': '', 'page': 4, 'title': ''}, page_content='1040-50.\n7.\nLauron S, Plasse C, Vaysset M, Pereira B, D’Incan M, Rondepierre F, et al.\nPrevalence and odds of depressive and anxiety disorders and symptoms in\nchildren and adults with alopecia areata: a systematic review and metaanalysis. JAMA Dermatol. 2023; 159(3): 281-8.\n8.\nKurd SK, Troxel AB, Crits-Christoph P, Gelfand JM. The risk of depression,\nanxiety, and suicidality in patients with psoriasis: a population-based cohort\nstudy. Arch Dermatol. 2010; 146(8): 891-5.\n9.\n

## 17. 질의응답 실행 (이미지 포함)

**이미지 분석 → 쿼리 재구성 → RAG 검색 → 답변 생성**

1. 이미지를 OpenAI Vision API로 먼저 분석
2. 이미지 분석 결과 + 사용자 질문을 합쳐서 검색 쿼리 생성
3. RAG 검색 수행
4. 최종 답변 생성 (텍스트만)

In [ ]:
# 이미지 경로와 질문을 수정하세요
image_path = "1.jpg"
query = "지금 머리가 이모냥인데 어떻게해야해?"

if Path(image_path).exists():
    print("=" * 50)
    print(f"이미지: {image_path}")
    print(f"질문: {query}")
    print("=" * 50)
    
    # 1단계: 이미지 인코딩
    print("\n[1단계] 이미지 인코딩 중...")
    image_base64 = encode_image(image_path)
    print("✅ 이미지 인코딩 완료")
    
    # 2단계: 이미지 분석 및 쿼리 생성 (통합 함수 사용)
    print("\n[2단계] 이미지 분석 및 검색용 쿼리 생성 중... (OpenAI Vision API 호출)")
    analysis_start = time.time()
    enhanced_query = analyze_image_and_create_query(image_base64, query)
    analysis_end = time.time()
    print(f"✅ 이미지 분석 및 쿼리 생성 완료: {analysis_end - analysis_start:.2f}초")
    print("\n📝 검색용 쿼리 (미리보기):")
    print(enhanced_query[:300] + "...")
    
    # 3단계: RAG 검색 및 답변 생성
    print("\n[3단계] RAG 검색 및 답변 생성 중... (OpenAI API 호출)")
    invoke_start = time.time()
    
    result = qa_chain.invoke({"question": enhanced_query})
    
    invoke_end = time.time()
    print(f"✅ RAG 쿼리 완료: {invoke_end - invoke_start:.2f}초")
    print("=" * 50)
    
    print("\n📝 최종 답변:")
    print(result)
    
    # 전체 소요 시간
    total_time = (analysis_end - analysis_start) + (invoke_end - invoke_start)
    print("\n" + "=" * 50)
    print(f"⏱️ 전체 소요 시간: {total_time:.2f}초")
    print(f"  - 이미지 분석 및 쿼리 생성: {analysis_end - analysis_start:.2f}초")
    print(f"  - RAG 검색 및 답변: {invoke_end - invoke_start:.2f}초")
    print("=" * 50)
else:
    print(f"❌ 이미지 파일을 찾을 수 없습니다: {image_path}")

In [19]:
# 마지막 질문에 대한 참고 문서 확인
retrieved_docs = hybrid_multi_retriever.invoke(query)

print("📚 참고 문서:")
print("=" * 50)
for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n[문서 {i}]")
    print(doc.page_content)
    print("-" * 50)

📚 참고 문서:

[문서 1]
속상하거나 우울할 때 헤어스타일 바꾸는 것이감정 치유에 영향을
끼친다고 생각한다.
.245
.755
.630
나는 스타일 변화가 만족스러우면 기분전환이 되고 스트레스가
풀리는것 같다.
.178
.752
.597
의욕이 없고 짜증이 많아질 때 헤어스타일을 바꾸면 완화될거라
생각한다.
.242
.661
.496
고유치
2.899
2.560
설명변량(%)
28.989
25.601
누적변량(%)
28.989
54.590
신뢰도계수(Cronbach'sα)
.801
.780
<Table 2> Validity and Reliability of Cmotional Healing
--------------------------------------------------

[문서 2]
어스타일 관리는 자신의 개성을 표현하며 외적
이미지로 변화를 줄수있는 중요한 수단으로써
의자기표현방법중 하나로 보여지고 있다.
한효주의 여성 헤어스타일 결정 요인과 관련된 선행연구에서는 20~40대 여성의 헤어스타
일 인식도와 선호도 연구8)에서는 여성들의 헤
어이미지 선호도와 헤어스타일 연출성향의 선
호도를 구체적으로 알아보는 연구를 하였고, 김
미선의 헤어스타일 결정에 영향을 미치는 개인적 요인과 헤어태도에 관한 연구9)에서는 헤어
스타일 변화의도에 영향을 미친후 행동으로 어떻게 이어지는지에 대해 분석하였으며, 이영애
의 여대생의 일반적 특성이 헤어스타일 결정에
미치는 영향에 관한 연구10)에서는 헤어스타일
에대한 정보를 제공하고 헤어스타일 결정에 다양한 욕구를 충족시키는지에 대해 연구를 하였
으며, 강수희의 헤어스타일별 시술주기와 헤어
스타일 변화에 영향을 주는 요인분석에 관한연
구11)에서는 미용실 손님들의 헤어스타일 관찰을 통하여 헤어스타일 변화 주기와 관련 요인을
연구하였다.
하지만 MZ세대 여성을 중심으로 얼굴이미지와 헤어스타일의 분석에 대한 연구는 찾아보
기 어렵다. 따라서 본 연구는 디지털문화에 익
숙하며, 유행에 민감하게 반응하고, SNS를 통한핵심소

## 18. 각 Retriever의 유사도 점수 확인

각 retriever의 점수를 확인하고 RRF로 병합된 결과를 비교합니다.

In [20]:
# 예시 3: RRF 병합 결과와 점수 확인
test_query = "원형탈모와 자존감의 관계"

print("\n" + "=" * 80)
print(f"테스트 쿼리: {test_query}")
print("=" * 80)

# 1. Dense와 Sparse retriever로 문서 가져오기
dense_docs = [result['document'] for result in get_dense_retriever_with_scores(test_query, k=5)]
sparse_docs = [result['document'] for result in get_bm25_retriever_with_scores(test_query, k=5)]

# 2. RRF로 병합하고 점수 확인
merged_results = rrf_merge_with_scores(dense_docs, sparse_docs, k=60)

# 3. 결과 출력
print_rrf_results(merged_results, top_k=7)


테스트 쿼리: 원형탈모와 자존감의 관계

📊 RRF 병합 결과 (상위 7개):

[1] RRF Score: 0.0164
Content Preview: 11) 조영미.(2009). 중년남성의 자기효능감, 인지연령및 신체만족도가 외모관리행동에 미치는
영향. 전남대학교대학원, 석사학위논문, pp.
62-64.
12) 김수현.(2019). 중년남성의 직업스트레스와
심리적위기감의 관계에서 갱년기증후군과 노
후준비도의 매개효과. 영남대학교 대학원, 석
사학위논문, pp.81-84.
13) 정성훈.(2013). 중년...
Source: data/중년남성의 앞머리 변화가 심리적 스트레스에 미치는 효과.pdf
--------------------------------------------------------------------------------

[2] RRF Score: 0.0164
Content Preview: 헤어스타일 변화로 인한감정 치유와 회복 탄력성의 관계에서 자존감의 매개효과
59
(+)의 관계를 보였다. 2단계 검증에서 내적치유는 낙관성, 수용성, 긍정성에 유의한 영향을 미치는 것으로 나타났고 모두 정(+)의 관계를보
였다. 3단계 검증에서 독립 변인인 내적치유
요인이 통제된 상태에서 매개 변인인 자존감이
회복 탄력성의 하위 요인에 유의한 영향을 미치는...
Source: data/헤어스타일 변화로 인한 감정 치유와 회복 탄력성의 관계에서 자존감의 매개 효과.pdf
--------------------------------------------------------------------------------

[3] RRF Score: 0.0161
Content Preview: 11) 조영미.(2009). 중년남성의 자기효능감, 인지연령및 신체만족도가 외모관리행동에 미치는
영향. 전남대학교대학원, 석사학위논문, pp.
62-64.
12) 김수현.(2019). 중년남성의 직업스트레스와
심리적위기감의 관계에서 갱년기증후군과 노
후준비도의 매개효과. 영남대학

In [21]:
# 예시 2: get_all_retrievers_with_scores 함수로 한 번에 확인
test_query = "탈모 치료 방법"

results = get_all_retrievers_with_scores(test_query, k=5)

테스트 쿼리: 탈모 치료 방법

📊 1. Dense Retriever (Vector Similarity):
--------------------------------------------------------------------------------
[1] Score: 0.3177 | Preview: 닥에서 네가닥으로 만들어져 있는증모 피스를 연결해 주는 방식이다. 말기 단계(Ludwig 제4단계)의 탈모증 또는
연모화로 너무얇은 모발에는 시술이 불가하다.
증모술에 사용하는 피...
[2] Score: 0.3177 | Preview: 닥에서 네가닥으로 만들어져 있는증모 피스를 연결해 주는 방식이다. 말기 단계(Ludwig 제4단계)의 탈모증 또는
연모화로 너무얇은 모발에는 시술이 불가하다.
증모술에 사용하는 피...
[3] Score: 0.3177 | Preview: 닥에서 네가닥으로 만들어져 있는증모 피스를 연결해 주는 방식이다. 말기 단계(Ludwig 제4단계)의 탈모증 또는
연모화로 너무얇은 모발에는 시술이 불가하다.
증모술에 사용하는 피...
[4] Score: 0.3177 | Preview: 닥에서 네가닥으로 만들어져 있는증모 피스를 연결해 주는 방식이다. 말기 단계(Ludwig 제4단계)의 탈모증 또는
연모화로 너무얇은 모발에는 시술이 불가하다.
증모술에 사용하는 피...
[5] Score: 0.3177 | Preview: 닥에서 네가닥으로 만들어져 있는증모 피스를 연결해 주는 방식이다. 말기 단계(Ludwig 제4단계)의 탈모증 또는
연모화로 너무얇은 모발에는 시술이 불가하다.
증모술에 사용하는 피...


📊 2. BM25 Retriever (Keyword-based):
--------------------------------------------------------------------------------
[1] Score: 5.0000 | Preview: 피크노제놀 성분이 한국 10~20대 여드름 피부에 미치는 영향489
께 자발적으로

In [22]:
# 예시 1: 각 retriever의 점수를 개별적으로 확인
test_query = "헤어 염색이 모발에 미치는 영향"

print("=" * 80)
print(f"테스트 쿼리: {test_query}")
print("=" * 80)

# Dense Retriever (벡터 유사도) 결과
print("\n📊 1. Dense Retriever (Vector Similarity) 결과:")
print("-" * 80)
dense_results = get_dense_retriever_with_scores(test_query, k=5)
for i, result in enumerate(dense_results, 1):
    print(f"\n[{i}] Score: {result['score']:.4f}")
    print(f"Content Preview: {result['document'].page_content[:150]}...")
    print(f"Source: {result['document'].metadata.get('source', 'N/A')}")

# BM25 Retriever (키워드 기반) 결과
print("\n\n📊 2. BM25 Retriever (Keyword-based) 결과:")
print("-" * 80)
sparse_results = get_bm25_retriever_with_scores(test_query, k=5)
for i, result in enumerate(sparse_results, 1):
    print(f"\n[{i}] Score: {result['score']:.4f}")
    print(f"Content Preview: {result['document'].page_content[:150]}...")
    print(f"Source: {result['document'].metadata.get('source', 'N/A')}")

테스트 쿼리: 헤어 염색이 모발에 미치는 영향

📊 1. Dense Retriever (Vector Similarity) 결과:
--------------------------------------------------------------------------------

[1] Score: 0.2541
Content Preview: 1. 서론
외모가 경쟁력이 되는현대 사회에서모발의 염색이
나 탈색을 통해 자신의 이미지와 개성을 표출하는 사람들이 증가하고 있다[1]. 모발은 모표피, 모피질, 모수질
의 구조로 되어 있으며, 모발의 가장바깥 부분에 위치한 모표피는 여러겹의 케라틴이 비늘 모양으로 겹쳐...
Source: data/시어버터가 함유된 헤어트리트먼트가 탈색 모발에 미치는 영향.pdf

[2] Score: 0.2541
Content Preview: 1. 서론
외모가 경쟁력이 되는현대 사회에서모발의 염색이
나 탈색을 통해 자신의 이미지와 개성을 표출하는 사람들이 증가하고 있다[1]. 모발은 모표피, 모피질, 모수질
의 구조로 되어 있으며, 모발의 가장바깥 부분에 위치한 모표피는 여러겹의 케라틴이 비늘 모양으로 겹쳐...
Source: data/시어버터가 함유된 헤어트리트먼트가 탈색 모발에 미치는 영향.pdf

[3] Score: 0.2541
Content Preview: 1. 서론
외모가 경쟁력이 되는현대 사회에서모발의 염색이
나 탈색을 통해 자신의 이미지와 개성을 표출하는 사람들이 증가하고 있다[1]. 모발은 모표피, 모피질, 모수질
의 구조로 되어 있으며, 모발의 가장바깥 부분에 위치한 모표피는 여러겹의 케라틴이 비늘 모양으로 겹쳐...
Source: data/시어버터가 함유된 헤어트리트먼트가 탈색 모발에 미치는 영향.pdf

[4] Score: 0.2541
Content Preview: 1. 서론
외모가 경쟁력이 되는현대 사회에서모발의 염색이
나 탈색을 통해 자신의 이미지와 개성을 표출하는 사람들이 증가하고 있다[1]. 모발은 모표피, 모피질, 모